In [1]:
print("Hello world")

Hello world


In [16]:
import tweepy
from deep_translator import GoogleTranslator
token = "AAAAAAAAAAAAAAAAAAAAAMREhAEAAAAAUyTLCpMhUzUtvTAH%2FQBGnduHwhU%3Dqrf0rs2d2j891MmZr2SGQKkPonhsjI5fNrAYM2GHYxp9Rw46Lm" #bearer token, toegang als developer
client = tweepy.Client(bearer_token=token)

userid = "7174972" #twitter id, converter
paginator = tweepy.Paginator(client.get_users_tweets, id=userid, max_results=8) #https://docs.tweepy.org/en/stable/client.html
translated = GoogleTranslator(source='auto', target='en').translate(text="buenas noches")
print(translated)

for tweet in paginator.flatten(limit=8): #Acht resultaten
    print(tweet)


good night
Chinese ambassade Den Haag bij betoging beklad met rode verf https://t.co/kgN1kiwiDQ
Goederentrein met veel auto's in brand bij Etten-Leur https://t.co/DTHKubnnQ9
Nederlaag voor online supers: personeel moet weekendtoeslag krijgen https://t.co/vRtPVxCyox
Vrouw valt bank in Beiroet binnen om spaargeld voor zieke zus op te nemen https://t.co/5qldx5RxAa
Cardiologen krijgen zonder toestemming miljoenen euro's van medische bedrijven https://t.co/KI9z8j8YUO
'Jumbo-topman Van Eerd vast als verdachte in witwaszaak' https://t.co/tHt1bCmKJT
Londen bereidt zich voor op afscheid van koningin Elizabeth https://t.co/5rxhMwQ6va
Donny M. bekent tijdens emotionele zitting ontvoeren, misbruiken en doden van 9-jarige Gino https://t.co/Ctef8OQgGL
